![eodag logo](https://eodag.readthedocs.io/en/latest/_static/eodag_logo.png)

# Eodag search plugin user manual

The eodag-labextension JupyterLab plugin brings satellite imagery search ability. With that extension you can search products, see results and check if results suits to your needs and finally generate Python code allowing to access to those products in the notebook.

That tool is based on [eodag](https://eodag.readthedocs.io).

## Configuration

For some providers we don’t need to set up credentials to search for products. But if you wish to download them, you should set the credentials beforehand, using these two environment variables for instance.

In [ ]:
# Example for PEPS
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__USERNAME"] = "PLEASE_CHANGE_ME"
os.environ["EODAG__PEPS__AUTH__CREDENTIALS__PASSWORD"] = "PLEASE_CHANGE_ME"

You can always configure eodag with your code, as shown in the [Configuration](https://eodag.readthedocs.io/en/stable/notebooks/api_user_guide/3_configuration.html) page of eodag's [Python API User Guide](https://eodag.readthedocs.io/en/stable/api_user_guide.html).

## Search

![extension logo](./images/eodag_labext_icon.png) Activate the plugin tab on the left of the JupuytuerLab interface by clicking on that icon.

![extension form](./images/eodag_labext_form.png)

With displayed search form, you can enter search extent and following search criteria:

* **Product type**: the searched product type. For each entry of the drop-down list, a tooltip is displayed at hovering time with corresponding description. 
* **Start date**: minimal date of the search temporal window.
* **End date**: maximal date of the search temporal window.
* **Max cloud cover**: maximum cloud cover allowed in search results in percent.
* **Additional parameters**: used to enter key-value pairs criteria for the request.

You can draw multiple extents, or use none. Each extent can be a rectangle or a free polygon.
Product type is mandatory. Other criterias are optional.

Once search criteria are filled out, click on the "Search" button to proceed to next step. At the end of the search, a popup opens and displays results.

## Results display

![extension popup](./images/eodag_labext_popup.png)

The results display popup is compopsed of 3 parts:

* a map showing products extent,
* a table listing products,
* a pane containing metadata for currently selected product.

The results table allows you to access product metadata by clicking on the desired product line. The magnifying glass button allows you to zoom in on the product's in the map view. By scrolling down in the list of results, the search is automatically restarted to retrieve the following results.

In the metadata view, clicking on the thumbnail displays it in native resolution. Clicking it again reduces its size again.

## Apply to the Jupyter notebook

If the search result is correct, clicking on the "Apply" button will insert the Python eodag code in a new cell of the currently open notebook. The popup is automatically closed. From there, it is possible to work in the notebook on the search results by executing the eodag search.

Here is an example of generated code:

In [ ]:
from eodag import EODataAccessGateway

dag = EODataAccessGateway()
product_type = "S2_MSI_L1C"
footprint = {
    "lonmin": 0.660957,
    "latmin": 43.149093,
    "lonmax": 2.388008,
    "latmax": 44.190082,
}
cloud_cover = 15
start, end = "2019-02-01", "2019-02-15"
search_results, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    geometry=footprint,
    startTimeFromAscendingNode=start,
    completionTimeFromAscendingNode=end,
    cloudCover=cloud_cover,
)

The eodag configuration file should be located at `$HOME/.config/eodag/eodag.yml`. See [eodag documentation](https://eodag.readthedocs.io/en/latest/intro.html#how-to-configure-authentication-for-available-providers) for further informations.

You may want to enforce usage of a particular provider. To do so, use `set_preferred_provider` and then execute a query like previously:

In [ ]:
dag = EODataAccessGateway()
dag.set_preferred_provider("theia")

## Using results

Here are some examples about how to use search results into a notebook:

In [ ]:
from pprint import pprint

# Display results list
pprint(search_results)

# Display products access paths
pprint([p.location for p in search_results])

### Extract products extent

In [ ]:
from eodag.api.search_result import SearchResult

results_geojson = SearchResult(search_results).as_geojson_object()

from shapely.geometry import shape, GeometryCollection

features = results_geojson["features"]
features = GeometryCollection(
    [shape(feature["geometry"]).buffer(0) for feature in features]
)
features

### Display products extent on a slippy map

In [ ]:
from folium import Map, GeoJson, Figure

ext = features.bounds
bounds = [[ext[1], ext[0]], [ext[3], ext[2]]]
m = Map(tiles="Stamen Terrain", control_scale=True,)
GeoJson(results_geojson).add_to(m)
m.fit_bounds(bounds)
Figure(width=500, height=300).add_child(m)

## Downloading products

To download products from the search request into a sub-directory called `download`, run:

In [ ]:
dag.download_all(search_results)

## Verbosity

Eodag verbosity can be increased with following call with levels:

- INFO: `verbose=1`
- DEBUG: `verbose=3`

In [ ]:
from eodag.utils.logging import setup_logging
setup_logging(verbose=3)

See [eodag documentation](https://eodag.readthedocs.io/) for more readings and tutorials about eodag library.